In [ ]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array
import pandas as pd

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)
 

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load dataset
filename = "/content/drive/My Drive/Latin Translation ML project/CData.csv"

In [ ]:
dataset_original = pd.read_csv(filename,converters={i: str for i in range(0, 2)})
dataset_original

,great,māgnus
0,great,māgna
1,great,māgnum
2,his own,suus
3,her own,sua
4,its own,suum
...,...,...
17439,"While I breathe, I hope","Dum spiro, spero"
17440,"While we have the time, let us do good","Dum tempus habemus, operemur bonum"
17441,"While life is, hope is",Dum vita est spes est
17442,"While we live, let us live","Dum vivimus, vivamus"


In [ ]:
clean_pairs= dataset_original.to_numpy()

In [ ]:

# save clean pairs to file
save_clean_data(clean_pairs, 'english-german.pkl')
# spot check
for i in range(100):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

Saved: english-german.pkl
[great] => [māgna]
[great] => [māgnum]
[his own] => [suus]
[her own] => [sua]
[its own] => [suum]
[other] => [alius]
[another] => [alius]
[other] => [alia]
[another] => [alia]
[other] => [aliud]
[another] => [aliud]
[much] => [multus]
[many] => [multus]
[much] => [multa]
[many] => [multa]
[much] => [multum]
[many] => [multum]
[your] => [tuus]
[your] => [tua]
[your] => [tuum]
[not any] => [nūllus]
[no one] => [nūllus]
[not any] => [nūlla]
[no one] => [nūlla]
[not any] => [nūllum]
[no one] => [nūllum]
[one] => [ūnus]
[one] => [ūna]
[one] => [ūnum]
[good] => [bonus]
[good] => [bona]
[good] => [bonum]
[whole] => [tōtus]
[entire] => [tōtus]
[whole] => [tōta]
[entire] => [tōta]
[whole] => [tōtum]
[entire] => [tōtum]
[first] => [prīmus]
[first] => [prīma]
[first] => [prīmum]
[situated above] => [superus]
[upper] => [superus]
[situated above] => [supera]
[upper] => [supera]
[situated above] => [superum]
[upper] => [superum]
[so great] => [tantus]
[so much] => [tantus]

In [ ]:
clean_pairs

array([['great', 'māgna'],
       ['great', 'māgnum'],
       ['his own', 'suus'],
       ...,
       ['While life is, hope is', 'Dum vita est spes est'],
       ['While we live, let us live', 'Dum vivimus, vivamus'],
       ['The law is harsh, but it is the law', 'Dura lex, sed lex']],
      dtype=object)

In [ ]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle
 
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('english-german.pkl')
# random shuffle
shuffle(raw_dataset)
# reduce dataset size
n_sentences = 12000
dataset = raw_dataset[:n_sentences]
# split into train/test
train, test = dataset[:10500], dataset[10500:]
# save
save_clean_data(dataset, 'english-german-both.pkl')
save_clean_data(train, 'english-german-train.pkl')
save_clean_data(test, 'english-german-test.pkl')

Saved: english-german-both.pkl
Saved: english-german-train.pkl
Saved: english-german-test.pkl


In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
 

In [ ]:

 
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)
 
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X
 
# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y
 
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model
 
# load datasets
dataset = load_clean_sentences('english-german-both.pkl')
train = load_clean_sentences('english-german-train.pkl')
test = load_clean_sentences('english-german-test.pkl')
 
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))
 

English Vocabulary Size: 4472
English Max Length: 35
German Vocabulary Size: 7270
German Max Length: 19


In [ ]:
# prepare training data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)

In [ ]:
# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize defined model
print(model.summary())
plot_model(model, to_file='mode2.png', show_shapes=True)
# fit model
filename = 'model2.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 256)           1380096   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 35, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 35, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 35, 3374)          867118    
Total params: 3,297,838
Trainable params: 3,297,838
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 10500 samples, validate on 1500 samples
Epoch 1/100
 - 144s - loss: 1.4681 - acc: 0.9037 - val_loss: 0.7724 - val_acc: 0.9170

Epoch 00001: val_loss improved from inf to 0.77237, saving model to model2.h5
Epoch 2/100
 - 142s - loss: 0.6889 - acc: 0.9166 - val_loss: 0.6437 - val_acc: 0.9182

Epoch 00002: val_loss improved from 0.77237 to 0.64368, saving model to model2.h5
Epoch 3/100
 - 144s - loss: 0.6134 - acc: 0.9188 - val_loss: 0.6092 - val_acc: 0.9194

Epoch 00003: val_loss improved from 0.64368 to 0.60916, saving model to model2.h5
Epoch 4/100
 - 142s - loss: 0.5901 - acc: 0.9204 - val_loss: 0.5933 - val_acc: 0.9217

Epoch 00004: val_loss improved from 0.60916 to 0.59327, saving model to model2.h5
Epoch 5/100
 - 143s - loss: 0.5673 - acc: 0.9228 - val_loss: 0.5817 - val_acc: 0.9220

Epoch 00005: val_loss improved from 0.59327 to 0.58173, saving model to model2.h5
Epoch 6/100
 - 142s - loss: 0.5544 - acc: 0.9230 - val_loss: 0.5742 - val_acc: 0.9221

Epoch 00006: val_loss i

In [ ]:
!cp model2.h5 drive/My\ Drive/

cp: cannot stat 'model2.h5': No such file or directory


In [ ]:
import numpy as np
tk= Tokenizer()

In [ ]:

from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
model_path = '/content/drive/My Drive/model2.h5'

In [ ]:
model = load_model(model_path)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 

In [ ]:
preds = model.predict_classes(testX.reshape((testX.shape[0],testX.shape[1])))

ValueError: ignored

In [ ]:
preds_text = []
for i in preds:
       temp = []
       for j in range(len(i)):
            t = word_for_id(i[j], eng_tokenizer)
            if j > 0: 
                if (t == word_for_id(i[j-1], eng_tokenizer)) or (t == None):
                     temp.append('')
                else:
                     temp.append(t)
            else:
                   if(t == None):
                          temp.append('')
                   else:
                          temp.append(t) 

       preds_text.append(' '.join(temp))

In [ ]:
pred_df = pd.DataFrame({'actual' : test[:,0], 'predicted' : preds_text})
pred_df.sample(30)


,actual,predicted
50,fate,fate
937,address,speech
549,fame,fame
263,O valors,virtues
988,"your death, my life",death is death we with us ...
212,the deep,the open sea
732,sheep,cattle
1163,try,try
1053,be hidden,lie hidden
1006,perhaps,perhaps


In [ ]:
Input_latin = ['Atque haec quidem quaestio communis est omnium philosophorum']
source = encode_sequences(ger_tokenizer,ger_length,Input_latin)
predict_sequence(model, eng_tokenizer, source)

'moreover the subject of this inquiry is the common property of all philosophers'

In [ ]:
Input_latin = ['atque']
source = encode_sequences(ger_tokenizer,ger_length,Input_latin)
predict_sequence(model, eng_tokenizer, source)

'and'

In [ ]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
 
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
 
# load datasets
dataset = load_clean_sentences('english-latin-both.pkl')
train = load_clean_sentences('english-latin-train.pkl')
test = load_clean_sentences('english-latin-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
lat_tokenizer = create_tokenizer(dataset[:, 1])
lat_vocab_size = len(lat_tokenizer.word_index) + 1
lat_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(lat_tokenizer, lat_length, train[:, 1])
testX = encode_sequences(lat_tokenizer, lat_length, test[:, 1])
 
# load model
model = load_model('model2.h5')
# test on some training sequences
print('train')
evaluate_model(model, lat_tokenizer, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, lat_tokenizer, testX, test)

NameError: ignored